# ⚙️ Configuration

In [ ]:
# pip install --upgrade  domolibrary
# pip install sqlparse
# pip install sql-metadata

## ⚙️ Auth Object
Configure a `DomoAuth` object that will be used to interact with `DomoDatasets`

For this project we must use `DomoFullAuth` because the bootstrap API will not accept token auth

In [ ]:
import os

domo_instance = "domo-community"
domo_username = os.environ["DOMO_USERNAME"]
domo_password = os.environ["DOJO_PASSWORD"]
sql_dataflow_id = 70

In [ ]:
import domolibrary.client.DomoAuth as dmda


async def generate_auth(domo_instance, domo_username, domo_password) -> dmda.DomoAuth:
    """receive auth credentials, generate and test an auth object"""

    print(f"⚙️ generate_auth:  retrieving config data from {domo_instance} ⚙️")

    auth = dmda.DomoFullAuth(
        domo_username=domo_username,
        domo_password=domo_password,
        domo_instance=domo_instance,
    )

    # confirm we can get a valid auth token
    await auth.get_auth_token()

    if not auth.token:
        raise Exception("💀 generate_auth: Error retrieving Auth Token")

    if auth.token:
        print("🌼 generate_auth:  success retrieving Auth Token")

    return auth


# test
test_auth = await generate_auth(
    domo_instance=domo_instance,
    domo_password=domo_password,
    domo_username=domo_username,
)

⚙️ generate_auth:  retrieving config data from domo-community ⚙️
🌼 generate_auth:  success retrieving Auth Token


## ⚙️ Dataflow Object
Configure a `DomoDataflow` 

In [ ]:
import domolibrary.classes.DomoDataflow as dmdf


async def generate_dataflow(dataflow_id: str, auth: dmda.DomoAuth):
    """generates a DomoDataset class object from a dataset_id"""

    return await dmdf.DomoDataflow.get_from_id(dataflow_id=dataflow_id, auth=auth)


# test
df = await generate_dataflow(dataflow_id=sql_dataflow_id, auth=test_auth)

## 💾 Process Actions

`DomoDataflow_Action` definitions are embeded in the `DomoDataflow.actions` property.  

The `DomoDataflow_Action` class only captures a portion of the available properties across all the different action types, so this can be extended as needed.

SQL statements are captured in `DomoDataflow_Action.sql` and can therefore be parsed or reformatted for presentation in other documentation tools.  In the example below we use the [sqlparse(https://github.com/andialbrecht/sqlparse]) and [sql_metadata(https://github.com/macbre/sql-metadata)] library to parse and present a dataflow.

In [ ]:
import sqlparse
import sql_metadata as sqlmd
import domolibrary.classes.DomoBootstrap as dmbsr
from pprint import pprint


def process_actions(domo_dataflow: dmdf.DomoDataflow, debug_api: bool = False):
    """exctract SQL from actions"""

    domo_actions = domo_dataflow.actions

    if not domo_actions:
        return

    clean_sql = []

    for action in domo_actions:
        sql = action.sql

        # not all domo actions will contain sql.
        if not sql:
            continue

        # alter according to your needs
        clean_sql.append(
            {
                "transform_sql": sqlparse.format(
                    sql, reindent_aligned=True, keyword_case="upper"
                ),
                "referenced_tables": ", ".join(sqlmd.Parser(sql).tables),
                "target_table": action.name,
            }
        )

    return clean_sql


res = process_actions(domo_dataflow=df)
pprint(res)

[{'referenced_tables': 'domostats_people',
  'target_table': 't_1',
  'transform_sql': 'SELECT `Display Name`,\n'
                   '       `User ID` , @row := @row+1 AS id , @isReset := CASE '
                   'WHEN @iter = @rowCounter THEN 1\n'
                   '            ELSE 0\n'
                   '             END isReset , @group := @isReset + @group AS '
                   'rowGroup , @iter := CASE WHEN @isReset = 1 THEN 1\n'
                   '            ELSE @iter + 1\n'
                   '             END iter , @rowCounter := CASE WHEN @isReset '
                   '= 1 THEN FLOOR(RAND() * (7-3) + 3)\n'
                   '            ELSE @rowCounter\n'
                   '             END numChildren\n'
                   '  FROM `domostats_people` p ,\n'
                   '       (\n'
                   '        SELECT @row:= 0,@rowCounter:= 1, @iter := 1, '
                   '@isReset := 0 , @group := 0\n'
                   '       ) r'},
 {'referenced_tabl

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()